In [0]:
%pip install -qqqq -U databricks-sdk
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("cyber_full_path", "", "cyber_full_path")
dbutils.widgets.text("user_full_path", "", "user_full_path")
dbutils.widgets.text("catalog", "", "catalog")
dbutils.widgets.text("schema", "", "schema")

In [0]:
def get_full_path(full_path):
    if full_path.startswith("/Workspace"):
        full_path = f"file:{full_path}"
    return full_path


cyber_full_path = get_full_path(dbutils.widgets.get("cyber_full_path"))
user_full_path = get_full_path(dbutils.widgets.get("user_full_path"))

In [0]:
cyber_threat_detection_df = spark.read.format("parquet").load(cyber_full_path)
user_info_df = spark.read.format("parquet").load(user_full_path)

In [0]:
cyber_threat_detection_df.display()

In [0]:
user_info_df.display()

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
cyber_threat_table_name = f"{catalog}.{schema}.cyber_threat_detection"
user_info_table_name = f"{catalog}.{schema}.user_info"
cyber_threat_detection_df.writeTo(cyber_threat_table_name).createOrReplace()
user_info_df.writeTo(user_info_table_name).createOrReplace()

In [0]:
%sql
CREATE OR REPLACE FUNCTION catalog.schema.get_cyber_threat_info(
    threat_type STRING COMMENT 'input cyber threat type'
  )
  RETURNS STRING
  COMMENT 'Returns latest threat_id, event_timestamp, source_ip, protocol, detection_tool given a threat_type'
  RETURN
    SELECT
      CONCAT(
        'Threat ID: ',
        threat_id,
        ', ',
        'Timestamp: ',
        event_timestamp,
        ', ',
        'Source IP: ',
        source_ip,
        ', ',
        'Protocol: ',
        protocol,
        ', ',
        'Detection Tool: ',
        detection_tool
      )
    FROM
      catalog.schema.cyber_threat_detection
    WHERE
      threat_type = threat_type
    ORDER BY
      event_timestamp DESC
    LIMIT 1;

In [0]:
%sql
-- SELECT
--   bo_cheng_dnb_demos.agents.get_cyber_threat_info("Ransomware")

In [0]:
%sql
CREATE OR REPLACE FUNCTION catalog.schema.get_user_info(
    source_ip STRING COMMENT 'input ip address'
  )
  RETURNS STRING
  COMMENT 'Returns latest user_name, department, email, ip_address, location given a source_ip address'
  RETURN
    SELECT
      CONCAT(
        'Username: ',
        user_name,
        ', ',
        'Department: ',
        department,
        ', ',
        'Email: ',
        email,
        ', ',
        'IP Address: ',
        ip_address,
        ', ',
        'Location: ',
        location
      )
    FROM
      catalog.schema.user_info
    WHERE
      ip_address = source_ip
    LIMIT 1;

In [0]:
%sql
-- SELECT
-- catalog.schema.get_user_info("192.168.1.21")